In [6]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import csv
import warnings 
warnings.filterwarnings('ignore')
from powergenome.util import reverse_dict_of_lists
from powergenome.price_adjustment import inflation_price_adjustment

In [9]:
#code from https://github.com/PowerGenome/PowerGenome/blob/e1549ca7495b077500c3081c1efa94ab74a146bd/example_system/test_settings.yml#L89-L118
cost_multiplier_region_map = {
    "TRE": ["ERC_PHDL", "ERC_REST", "ERC_WEST"],
    "FRCC": ["FRCC"],
    "MISW": ["MIS_WUMS", "MIS_MNWI", "MIS_IA"],
    "MISE": ["MIS_LMI"],
    "PJMC": ["PJM_COMD"],
    "MISC": ["MIS_IL", "MIS_MO", "S_D_AECI", "MIS_INKY"],
    "SPPN": ["MIS_MAPP", "SPP_WAUE", "SPP_NEBR", "MIS_MIDA"],
    "SPPC": ["SPP_N"],
    "SPPS": ["SPP_WEST", "SPP_SPS"],
    "MISS": ["MIS_AMSO", "MIS_WOTA", "MIS_LA", "MIS_AR", "MIS_D_MS"],
    "SRSE": ["S_SOU"],
    "SRCA": ["S_VACA"],
    "PJMD": ["PJM_Dom"],
    "PJMW": ["PJM_West", "PJM_AP", "PJM_ATSI"],
    "PJME": ["PJM_WMAC", "PJM_EMAC", "PJM_SMAC", "PJM_PENE"],
    "SRCE": ["S_C_TVA", "S_C_KY"],
    "NYUP": ["NY_Z_A", "NY_Z_B", "NY_Z_C&E", "NY_Z_D", "NY_Z_F", "NY_Z_G-I",],
    "NYCW": ["NY_Z_J", "NY_Z_K"],
    "ISNE": ["NENG_ME", "NENGREST", "NENG_CT"],
    "RMRG": ["WECC_CO"],
    "BASN": ["WECC_ID", "WECC_WY", "WECC_UT", "WECC_NNV"],
    "NWPP": ["WECC_PNW", "WECC_MT"],
    "CANO": ["WEC_CALN", "WEC_BANC"],
    "CASO": ["WECC_IID", "WECC_SCE", "WEC_LADW", "WEC_SDGE"],
    "SRSG": ["WECC_AZ", "WECC_NM", "WECC_SNV"],
}
rev_cost_mult_region_map = reverse_dict_of_lists(cost_multiplier_region_map)

tx_capex_region_map = {
    "wecc": [
        "WECC_AZ",
        "WECC_CO",
        "WECC_ID",
        "WECC_MT",
        "WECC_NM",
        "WECC_NNV",
        "WECC_PNW",
        "WECC_SNV",
        "WECC_UT",
        "WECC_WY",
    ],
    "ca": ["WEC_BANC", "WEC_CALN", "WEC_LADW", "WEC_SDGE", "WECC_IID", "WECC_SCE",],
    "tx": ["ERC_PHDL", "ERC_REST", "ERC_WEST",],
    "upper_midwest": [
        "MIS_MAPP",
        "SPP_WAUE",
        "MIS_MNWI",
        "MIS_MIDA",
        "MIS_IA",
        "MIS_IL",
        "MIS_INKY",
    ],
    "lower_midwest": ["SPP_N", "SPP_WEST", "SPP_SPS", "SPP_NEBR",],
    "miso_s": [
        "MIS_LA",
        "MIS_WOTA",
        "MIS_AMSO",
        "MIS_AR",
        "MIS_MO",
        "S_D_AECI",
        "MIS_D_MS",
    ],
    "great_lakes": ["MIS_WUMS", "MIS_LMI",],
    "pjm_s": ["PJM_AP", "PJM_ATSI", "PJM_COMD", "PJM_Dom", "PJM_West", "S_C_KY",],
    "pj_pa": ["PJM_PENE", "PJM_WMAC",],
    "pjm_md_nj": ["PJM_EMAC", "PJM_SMAC"],
    "ny": ["NY_Z_A", "NY_Z_B", "NY_Z_C&E", "NY_Z_D", "NY_Z_F", "NY_Z_G-I", "NY_Z_J",],
    "tva": ["S_C_TVA",],
    "south": ["S_SOU",],
    "fl": ["FRCC"],
    "vaca": ["S_VACA"],
    "ne": ["NY_Z_K", "NENG_CT", "NENG_ME", "NENGREST",],
}

rev_region_mapping = reverse_dict_of_lists(tx_capex_region_map)

spur_costs_2013 = {
    "wecc": 3900,
    "ca": 3900 * 2.25,  # According to Reeds docs, CA is 2.25x the rest of WECC
    "tx": 3900,
    "upper_midwest": 3900,
    "lower_midwest": 3800,
    "miso_s": 3900 * 2.25,
    "great_lakes": 4100,
    "pjm_s": 3900 * 2.25,
    "pj_pa": 3900 * 2.25,
    "pjm_md_nj": 3900 * 2.25,
    "ny": 3900 * 2.25,
    "tva": 3800,
    "south": 4950,
    "fl": 4100,
    "vaca": 3800,
    "ne": 3900 * 2.25,
}

spur_costs_2018 = {
    region: inflation_price_adjustment(cost, 2013, 2018)
    for region, cost in spur_costs_2013.items()
}

tx_costs_2013 = {
    "wecc": 1350,
    "ca": 1350 * 2.25,  # According to Reeds docs, CA is 2.25x the rest of WECC
    "tx": 1350,
    "upper_midwest": 900,
    "lower_midwest": 900,
    "miso_s": 1750,
    "great_lakes": 1050,
    "pjm_s": 1350,
    "pj_pa": 1750,
    "pjm_md_nj": 4250,  # Bins are $1500 wide - assume max bin is $750 above max
    "ny": 2750,
    "tva": 1050,
    "south": 1350,
    "fl": 1350,
    "vaca": 900,
    "ne": 4250,  # Bins are $1500 wide - assume max bin is $750 above max
}

tx_costs_2018 = {
    region: inflation_price_adjustment(cost, 2013, 2018)
    for region, cost in tx_costs_2013.items()
}

In [18]:
map_cost_regions_to_temoa = dict()
map_cost_regions_to_temoa['CA'] = 'ca'
map_cost_regions_to_temoa['NW'] = 'wecc'
map_cost_regions_to_temoa['SW'] = 'wecc'
map_cost_regions_to_temoa['TX'] = 'tx'
map_cost_regions_to_temoa['NE'] = 'ny'
map_cost_regions_to_temoa['SE'] = 'tva'
map_cost_regions_to_temoa['MID_AT'] = 'pj_pa'
map_cost_regions_to_temoa['N_CEN'] = 'upper_midwest'
map_cost_regions_to_temoa['CEN'] = 'miso_s'


In [42]:
df_tx_spur_costs = pd.DataFrame(columns=['temoa_regions','transmission_regions','spur_costs', 'tx_costs'])
df_tx_spur_costs['temoa_regions'] = list(map_cost_regions_to_temoa.keys())
df_tx_spur_costs['transmission_regions'] = df_tx_spur_costs['temoa_regions'].map(map_cost_regions_to_temoa)
df_tx_spur_costs['spur_costs'] = df_tx_spur_costs['transmission_regions'].map(spur_costs_2018)
df_tx_spur_costs['tx_costs'] = df_tx_spur_costs['transmission_regions'].map(tx_costs_2018)

In [43]:
df_tx_spur_costs['spur_costs'] = df_tx_spur_costs['spur_costs'].map(int)
df_tx_spur_costs['tx_costs'] = df_tx_spur_costs['tx_costs'].map(int)

In [44]:
df_tx_spur_costs[['temoa_regions','spur_costs']].set_index('temoa_regions').to_dict()['spur_costs']

{'CA': 9458,
 'NW': 4203,
 'SW': 4203,
 'TX': 4203,
 'NE': 9458,
 'SE': 4096,
 'MID_AT': 9458,
 'N_CEN': 4203,
 'CEN': 9458}

In [45]:
df_tx_spur_costs[['temoa_regions','tx_costs']].set_index('temoa_regions').to_dict()['tx_costs']

{'CA': 3274,
 'NW': 1455,
 'SW': 1455,
 'TX': 1455,
 'NE': 2964,
 'SE': 1131,
 'MID_AT': 1886,
 'N_CEN': 970,
 'CEN': 1886}